In [ ]:
import tensorflow as tf
import sys, os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import imghdr
import splitfolders
from skimage.io import imread
from skimage.transform import resize
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from PIL import ImageFile
from PIL import Image
import random
ImageFile.LOAD_TRUNCATED_IMAGES = True

from PyQt5.QtWidgets import QApplication, QWidget, QLabel, QVBoxLayout, QPushButton, QGridLayout, QHBoxLayout
from PyQt5.QtCore import Qt
from PyQt5.QtGui import QPixmap, QFont

In [ ]:
from tensorflow import keras
import tensorflow
import keras
from tensorflow.keras.models import load_model

new_model = load_model( 'happySadModel.h5')

In [ ]:
new_model

In [ ]:
def classification(images):
    new_model = load_model('happySadModel.h5')
    img = cv2.imread(images)
    resize = tf.image.resize(img, (256, 256))
    return new_model.predict(np.expand_dims(resize/255, 0))

In [ ]:
class ImageLabel(QLabel):
    def __init__(self):
        super().__init__()

        self.setAlignment(Qt.AlignCenter)
        self.setFixedHeight(500)
        self.setText('\n\n Drop Image Here \n\n')
        self.setStyleSheet('''
        QLabel {
            border:4px dashed #aaa
            }
            ''')


    def setPixmap(self, image):
        super().setPixmap(image)

class AppDemo(QWidget):
    def __init__(self):
        super().__init__()
        self.resize(1000, 900)
        self.setAcceptDrops(True)
        self.photo = False

        mainLayout = QVBoxLayout()

        self.label = QLabel("This application will determine if the image is happy or sad. Please insert a jpeg image")
        self.label.setFont(QFont('Times font', 15))
        mainLayout.addWidget(self.label)

        self.photoViewer = ImageLabel()
        mainLayout.addWidget(self.photoViewer)

        self.buttons = QHBoxLayout()
        
        self.button2 = QPushButton("Clear")
        self.button2.clicked.connect(self.clearAction)
        
        self.exitButton = QPushButton("Exit")
        self.exitButton.clicked.connect(self.close)
        
        self.buttons.addWidget(self.button2)
        self.buttons.addWidget(self.exitButton)
        mainLayout.addLayout(self.buttons)

        self.setLayout(mainLayout)

    def dragEnterEvent(self, event):
        if event.mimeData().hasImage:
            event.accept()
        else:
            event.ignore()

    def dragMoveEvent(self, event):
        if event.mimeData().hasImage:
            event.accept()
        else:
            event.ignore()

    def dropEvent(self, event):
        if event.mimeData().hasImage:
            event.setDropAction(Qt.CopyAction)
            filePath = event.mimeData().urls()[0].toLocalFile()
            self.set_image(filePath)

            event.accept()
        else:
            event.ignore()

    def set_image(self, filePath):
        self.photo = True
        self.photoViewer.setPixmap(QPixmap(filePath))
        
        yhat = classification(filePath)
        self.updateText(yhat)
        
    
    def updateText(self, yhat):
        if yhat < 0.5:
            self.label.setText("This is a happy person")
            
        else:
            self.label.setText("This is a sad person")
            
        
    def clearAction(self):
        if self.photo == True:
            self.photoViewer.clear()
            self.photo = False
            self.label.setText("This application will determine if the image is happy or sad. Please insert a jpeg image")
        else:
            print("false")

app = QApplication(sys.argv)

demo = AppDemo()
demo.show()
sys.exit(app.exec())
